In [ ]:
import fiona
import os
import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt
from ipywidgets import HBox, Label, IntSlider,IntRangeSlider
from ipywidgets import HTML
from IPython.display import clear_output
from ipyleaflet import Map, basemaps, basemap_to_tiles, CircleMarker, Popup, Polyline, Polygon, LayerGroup, LayersControl,ScaleControl, FullScreenControl
from ipyleaflet import WidgetControl
from ipywidgets import interactive
import matplotlib as mpl
import ipywidgets as widgets
from ipywidgets import HBox, Label, IntSlider,IntRangeSlider
#import matplotlib.patches as patches
#from matplotlib.patches import Polygon
import matplotlib
import matplotlib.colors
import contextily as cx
import shapely.geometry as sg
from rijksdriehoek import rijksdriehoek
from matplotlib.collections import PatchCollection
import ipympl
import warnings
import numpy as np
import ast
import datetime as dt
import time as t
import pandas as pd
import glob


RD = rijksdriehoek.Rijksdriehoek()
plt.rcParams.update({'font.size': 24})
MAX_GROUP_SIZE = 250
GPKG_VIS_MAP = {}
if os.path.exists("Data/gpkg_vis_map.txt"):
    f = open("Data/gpkg_vis_map.txt")
    GPKG_VIS_MAP = eval(f.read())
    f.close()

In [ ]:
def read_KNMI_file(filename):
    f = open(filename)
    data = f.read().split('\n')
    f.close()
    start_idx = None
    for i in range(len(data)):
        if '# STN,YYYYMMDD' in data[i]:
            start_idx = i + 2
            break
    headers = data[start_idx-2].split(',')
    headers = [h.strip() for h in headers]
    evapo_id = headers.index('EV24')
    date_id = headers.index('YYYYMMDD')
    precip_id = headers.index('RH')
    temp_id = headers.index('TX')
    processed_data = {}
    for row in data[start_idx:]:
        if row != '':
            split_row = row.split(',')
            date = dt.datetime.strptime(split_row[date_id].strip(), '%Y%m%d')
            if '' not in [date, split_row[evapo_id].strip(), split_row[precip_id].strip(), split_row[temp_id].strip()]:
                if '348' in filename and date.year < 1988: # very incomplete data before 1988
                    continue
                processed_data[date] = [eval(split_row[precip_id].strip()),
                                        eval(split_row[evapo_id].strip()),
                                        eval(split_row[temp_id].strip())]
                if processed_data[date][0] == -1:
                    processed_data[date][0] = 0
                processed_data[date][0] /= 10
                processed_data[date][1] /= 10
                processed_data[date][2] /= 10
    return processed_data

def get_yearfrac(date):
    year = eval(date[:4])
    month = eval(date[5:7].lstrip('0'))
    day = eval(date[8:].lstrip('0'))
    dayOfYear = dt.datetime(year=year, month=month, day=day) - dt.datetime(year=year, month=1, day=1)
    yearDays = dt.datetime(year=year+1, month=1, day=1) - dt.datetime(year=year, month=1, day=1)
    yearfrac = year + dayOfYear.days / yearDays.days
    return yearfrac

In [ ]:
t0 = '2015-01-02'
Cabauw = read_KNMI_file(f'Data/etmgeg_348.txt')
tmax = max(Cabauw.keys())
tmax = '{:0>4d}-{:0>2d}-{:0>2d}'.format(tmax.year, tmax.month, tmax.day)
tmin = min(Cabauw.keys()) + dt.timedelta(days=365) # to give space for tau to run back
tmin = '{:0>4d}-{:0>2d}-{:0>2d}'.format(tmin.year, tmin.month, tmin.day)
etmgeg = Cabauw 

tmin_yf = get_yearfrac(tmin)
tmax_yf = get_yearfrac(tmax)
t0_yf = get_yearfrac(t0)
t0_dt = dt.datetime.strptime(t0, '%Y-%m-%d')
tmin_dt = dt.datetime.strptime(tmin, '%Y-%m-%d')
tmax_dt = dt.datetime.strptime(tmax, '%Y-%m-%d')

t_eval_min = dt.datetime(1945, 1, 1)

In [ ]:
def read_spams10(filename, savename="Data/spams10_delfland.txt"):
    print(f"Processing {filename}...")
    if os.path.exists(savename):
        f = open(savename)
        data = f.read()
        f.close()
        data_dict = eval(data)
    else:
        data_spams10 = pd.read_parquet(filename)
        data_dict = {}
        for idx in range(len(data_spams10[data_spams10.columns[0]])):
            data_dict[idx] = {}
            for col in data_spams10.columns:
                data_dict[idx][col] = data_spams10[col][idx]
            data_dict[idx]["pnt_crd"] = (data_dict[idx]["pnt_lon"], data_dict[idx]["pnt_lat"])
            RD.from_wgs(data_dict[idx]["pnt_lat"], data_dict[idx]["pnt_lon"])
            data_dict[idx]["center_dec"] = (int(RD.rd_x), int(RD.rd_y))
            data_dict[idx]["model_mode"] = "SPAMS_default"
            data_dict[idx]["has_InSAR"] = True
            data_dict[idx]["has_surfaceleveling"] = False
            
            for key in ['h2015_m', 'sigma_h2015_m', #'h2023_m', 'v_mpy', 'sigma_v_mpy', 
                       'xI_mpy', 'sigma_xI_mpy', 'xI_frac_1000pct', 
                       'mean_yearly_irreversible_subsidence_MYIS_mpy', 'sigma_MYIS_mpy',
                       'xP_mpmm', 'sigma_xP_mpmm', 'xE_mpmm', 'sigma_xE_mpmm', 'tau_kd', 'sigma_tau_kd',
                       'rho_h2015_xI_1000', 'rho_h2015_xP_1000', 'rho_h2015_xE_1000', 'rho_h2015_tau_1000',
                       'rho_xI_xP_1000', 'rho_xI_xE_1000', 'rho_xI_tau_1000', 
                       'rho_xP_xE_1000', 'rho_xP_tau_1000', 
                       'rho_xE_tau_1000',
                       'timespan_ky', 'n_observations_1000', 
                       'significant_irreversible_subsidence_detected_2s_1000', 'significant_irreversible_subsidence_detected_3s_1000', 
                       'discriminatory_power_2s_1000pct', 
                       'discriminatory_power_3s_1000pct', 'MDD_80p_mpy', 
                       'cum_irreversible_subsidence_since_2000_m', 'oldest_observation_ky', 
                       'OMT_sust_2s_1000', 'OMT_sust_3s_1000', 'F_value_norm']:
                if key in ["h2015_m", "sigma_h2015_m",'rho_h2015_xI_1000', 'rho_h2015_xP_1000', 'rho_h2015_xE_1000', 'rho_h2015_tau_1000',
                          'rho_xI_tau_1000', 'rho_xP_tau_1000', 'rho_xE_tau_1000', 'significant_irreversible_subsidence_detected_2s_1000', 'significant_irreversible_subsidence_detected_3s_1000',
                          'timespan_ky', 'discriminatory_power_2s_1000pct', 'discriminatory_power_3s_1000pct', 'MDD_80p_mpy',
                          'cum_irreversible_subsidence_since_2000_m', 'oldest_observation_ky',
                          'OMT_sust_2s_1000', 'OMT_sust_3s_1000', 'F_value_norm', 'sigma_tau_kd']:
                    data_dict[idx][key] = None
                elif key == "xI_mpy":
                    data_dict[idx][key] = data_dict[idx]["xI"] * 365.2425 / 1000
                elif key == "sigma_xI_mpy":
                    data_dict[idx][key] = np.sqrt(data_dict[idx]["var_xI"]) * 365.2425 / 1000
                elif key in ["xP_mpmm", "xE_mpmm", "tau_kd"]:
                    data_dict[idx][key] = data_dict[idx][key.split("_")[0]] / 1000
                elif key in ["sigma_xP_mpmm", "sigma_xE_mpmm"]:
                    data_dict[idx][key] = np.sqrt(data_dict[idx]["var_" + key.split("_")[1]]) / 1000
                elif key in ['rho_xI_xP_1000', 'rho_xI_xE_1000', 'rho_xP_xE_1000']:
                    build_key = "cov_"
                    mult = 1
                    if "xP" in key:
                        build_key += "xP"
                    if "xE" in key:
                        build_key += "xE"
                    if "xI" in key:
                        build_key += "xI"
                        mult = 365.2425
                    data_dict[idx][key] = data_dict[idx][build_key] * mult / 1e9 / np.sqrt(data_dict[idx]["var_" + key.split("_")[1]]) / np.sqrt(data_dict[idx]["var_" + key.split("_")[2]])
                elif key == "n_observations_1000":
                    data_dict[idx][key] = data_dict[idx]["dof"] + 3
                elif key == "xI_frac_1000pct":
                
                    ndays = (tmax_dt-tmin_dt).days
                
                    R_array = np.zeros((ndays, ))
                    t0_idx = None
                
                    for day in range(ndays):
                        for backwards_day in range(int(data_dict[idx]["tau"])+1):
                            cur_day = tmin_dt + dt.timedelta(days=day-backwards_day)
                            try:
                                Pt = etmgeg[cur_day][0]
                                Et = etmgeg[cur_day][1]
                                if backwards_day == int(data_dict[idx]["tau"]):
                                    R_array[day] += (Pt*data_dict[idx]["xP"] - Et*data_dict[idx]["xE"]) * (data_dict[idx]["tau"] % 1)
                                else:
                                    R_array[day] += (Pt*data_dict[idx]["xP"] - Et*data_dict[idx]["xE"])
                
                            except KeyError:
                                continue
                    data_dict[idx][key] = (sum(1*R_array<=0)/R_array.shape[0]) * 100 / 1000
                elif key == "mean_yearly_irreversible_subsidence_MYIS_mpy":
                    data_dict[idx][key] = data_dict[idx]["xI_frac_1000pct"] * 10 * data_dict[idx]["xI_mpy"]
                elif key == "sigma_MYIS_mpy":
                    data_dict[idx][key] = data_dict[idx]["xI_frac_1000pct"] * 10 * data_dict[idx]["sigma_xI_mpy"]
                else:
                    raise ValueError(f"Unhandled key {key}!")
            if idx % 10 == 0:
                print(f"{idx+1}/{len(data_spams10[data_spams10.columns[0]])}")
        f = open(savename, "w")
        f.write(str(data_dict))
        f.close()
    return data_dict


def add_vis_gpkg_parameters(vis_gpkg, gpkg_data):
    ct = 0
    for EUP in list(gpkg_data.keys()):
        txt_eup = map_index(EUP)
        vis_eup = map_vis_index(EUP)  
        if vis_eup is not None:
            if False:
                t, h = plot_model(txt_eup)
                y, yt, ys = get_data(txt_eup)
                EUP_f = open(f'Data/EUPs/{txt_eup}.txt')
                raw = EUP_f.read()
                EUP_f.close()
                raw_EUP = raw.replace('nan', "'nan'").replace('array', 'np.array')
                EUP_data = eval(raw_EUP)
        
                yhat = [h[t.index(yt_)] for yt_ in yt]
                ehat = np.array(yhat) - np.array(y)
                T_stat = np.sum(ehat ** 2 / np.array(ys) ** 2)
                if EUP_data["model_mode"] == "SPAMS":
                    nu = int(1000*vis_data[vis_eup]['n_observations_1000']) - 5
                    vis_gpkg[vis_eup]["has_InSAR"] = True
                else:
                    nu = int(1000*vis_data[vis_eup]['n_observations_1000']) - 2
                    vis_gpkg[vis_eup]["has_InSAR"] = False
                if nu == 0:
                    T_stat_norm = 1000000
                else:
                    T_stat_norm = T_stat / nu
                vis_gpkg[vis_eup]["F_value_norm"] = T_stat_norm

                if min(yt) < 1995:
                    vis_gpkg[vis_eup]["has_surfaceleveling"] = True
                else:
                    vis_gpkg[vis_eup]["has_surfaceleveling"] = False

            vis_gpkg[vis_eup]["center_dec"] = (int(np.mean([ii[0] for ii in vis_gpkg[vis_eup]["crd_dec"][0]])),
                                               int(np.mean([ii[1] for ii in vis_gpkg[vis_eup]["crd_dec"][0]])))


        ct += 1
        if ct % 1000 == 0:
            print(f"{ct}/{len(list(gpkg_data.keys()))}")
    return vis_gpkg
        

In [ ]:
def read_gpkg(filename):
    
    assert '.' in filename, f'Filename {filename} does not have an extension!'    
    if filename.split('.')[1] != 'gpkg':
        raise ValueError(f'Expected gpkg, got {filename}')
    if not os.path.isfile(f'Data/{filename}'):
        raise ValueError(f'File {filename} not found in Data directory!')
    
    AOI = {}
    with fiona.open(f"Data/{filename}") as layer:
        for feature in layer:     
            AOI[feature['properties']['ogc_fid']] = {
                "crd_dec": [feature['geometry']['coordinates'][ii] for ii in range(len(feature['geometry']['coordinates']))]
                }
            for key in ['h0', 's_h0', 'xI', 's_xI', 'xP', 's_xP', 'xE', 's_xE', 'tau', 's_tau',
                        'rho_xIh0', 'rho_xPh0', 'rho_xEh0', 'rho_tauh0',
                        'rho_xPxI', 'rho_xExI', 'rho_xItau', 
                        'rho_xPxE', 'rho_xPtau', 
                        'rho_xEtau', 'xI_frac']:
                AOI[feature['properties']['ogc_fid']][key] = feature['properties'][key]
    return AOI

def read_visualisation_gpkg(filename):
    assert '.' in filename, f'Filename {filename} does not have an extension!'    
    if filename.split('.')[1] != 'gpkg':
        raise ValueError(f'Expected gpkg, got {filename}')
    if not os.path.isfile(f'Data/{filename}'):
        raise ValueError(f'File {filename} not found in Data directory!')
    
    AOI = {}
    with fiona.open(f"Data/{filename}") as layer:
        for feature in layer:     
            AOI[feature['properties']['ogc_fid']] = {
                "crd_dec": [feature['geometry']['coordinates'][ii] for ii in range(len(feature['geometry']['coordinates']))]
                }
            for key in ['h2015_m', 'sigma_h2015_m', #'h2023_m', 'v_mpy', 'sigma_v_mpy', 
                   'xI_mpy', 'sigma_xI_mpy', 'xI_frac_1000pct', 
                   'mean_yearly_irreversible_subsidence_MYIS_mpy', 'sigma_MYIS_mpy',
                   'xP_mpmm', 'sigma_xP_mpmm', 'xE_mpmm', 'sigma_xE_mpmm', 'tau_kd', 'sigma_tau_kd',
                   'rho_h2015_xI_1000', 'rho_h2015_xP_1000', 'rho_h2015_xE_1000', 'rho_h2015_tau_1000',
                   'rho_xI_xP_1000', 'rho_xI_xE_1000', 'rho_xI_tau_1000', 
                   'rho_xP_xE_1000', 'rho_xP_tau_1000', 
                   'rho_xE_tau_1000',
                   'timespan_ky', 'n_observations_1000', 
                   'significant_irreversible_subsidence_detected_2s_1000', 'significant_irreversible_subsidence_detected_3s_1000', 
                   'discriminatory_power_2s_1000pct', 
                   'discriminatory_power_3s_1000pct', 'MDD_80p_mpy', 
                   'cum_irreversible_subsidence_since_2000_m', 'oldest_observation_ky', 
                   'OMT_sust_2s_1000', 'OMT_sust_3s_1000']:
                AOI[feature['properties']['ogc_fid']][key] = feature['properties'][key]
    return AOI

def convert_wgs84_to_rd(list_of_coords):
    """
    Converts a list of coordinates in WGS84 to RD
    :param list_of_coords: list of (lon, lat) objects
    :return: list of (rd_x, rd_y) objects
    """
    rd_coords = []
    for coord in list_of_coords:
        RD.from_wgs(lon=coord[0], lat=coord[1])
        rd_coords.append([RD.rd_x, RD.rd_y])
    return rd_coords

def convert_rd_to_wgs84(list_of_coords):
    """
    Converts a list of coordinates in RD to WGS
    :param list_of_coords: list of (rdx, rdy) objects
    :return: list of (lon, lat) objects
    """
    wgs_coords = []
    for coord in list_of_coords:
        RD.rd_x = coord[0]
        RD.rd_y = coord[1]
        wgs_coords.append(RD.to_wgs())
    return wgs_coords

In [ ]:
def read_aoi():
    filename = "Data/AoI_GreenHeart-polygon.shp"
    shape = fiona.open(filename)
    finished = False
    shape_iter = iter(shape)
    aoi_rd_poly = None
    while not finished:
        try:
            shp = next(shape_iter)
            polygons = [sg.Polygon(convert_wgs84_to_rd(shp["geometry"]["coordinates"][ii])) for ii in range(len(shp["geometry"]["coordinates"]))]
            if len(polygons) > 1 or aoi_rd_poly is not None:
                raise ValueError("Cannot handle AOI with more than 1 polygon!")
            aoi_rd_poly = polygons[0]
            if not aoi_rd_poly.is_valid:
                raise ValueError("Invalid polygon AOI!")
        except StopIteration:
            finished = True
    if aoi_rd_poly is None:
        raise ValueError(f"No valid AOI polygon found in {filename}!")
    return convert_rd_to_wgs84(np.array(aoi_rd_poly.exterior.coords.xy).T)
aoi_wgs_poly = read_aoi()

In [ ]:
def get_data(file):
    EUP_f = open(f'Data/EUPs/{file}.txt')
    raw = EUP_f.read()
    EUP_f.close()
    raw_EUP = raw.replace('nan', "'nan'").replace('array', 'np.array')
    EUP = eval(raw_EUP)

    if EUP['InSAR'] != -999:
        for keykey in [k for k in EUP['InSAR'].keys() if k[0] == 'h']:
            for d in range(len(EUP['InSAR'][keykey])):
                if EUP['InSAR'][keykey][d] == 'nan':
                    EUP['InSAR'][keykey][d] = np.nan

    
    y = []
    y_insar = []
    t = []
    t_insar = []
    s = []
    s_insar = []
    min_t = None
    max_t = None

    if EUP['EUP_type'] == 'brp':
        for key in ['100_AHN1', '100_AHN2', '100_AHN3', '100_AHN4', '_ThMD']:
            if EUP[f't{key}'] not in [-999, '']:
                y.append(EUP[f'h{key}'])
                t.append(EUP[f't{key}'])
                s.append(max(EUP[f's{key}'], 0.001))

        if EUP['InSAR'] != -999:
            keys = [key[1:] for key in EUP['InSAR'].keys() if key[0] == 't']
            for key in keys:
                for t_ in range(len(EUP['InSAR'][f't{key}'])):
                    if not np.isnan(EUP['InSAR'][f'h{key}'][t_]):
                        t_insar.append(EUP['InSAR'][f't{key}'][t_])
                        y_insar.append(EUP['InSAR'][f'h{key}'][t_])
                        s_insar.append(0.01)

    else:
        for key in ['100_AHN1', '100_AHN2', '100_AHN3', '100_AHN4', '_ThMD']:
            if EUP[f't{key}'] not in [-999, '']:
                y.append(EUP[f'h{key}'])
                t.append(EUP[f't{key}'])
                s.append(EUP[f's{key}'])
    full_y = []
    full_t = []
    full_s = []
    for d in range(len(y)):
        full_y.append(y[d])
        full_t.append(get_yearfrac(t[d]))
        full_s.append(s[d])
    for d in range(len(y_insar)):
        full_y.append(y_insar[d]+EUP['h0'])
        full_t.append(get_yearfrac(t_insar[d]))
        full_s.append(s_insar[d])
    return full_y, full_t, full_s

def plot_model(file, mode="DDEM"):
    if mode == "DDEM":
        EUP_f = open(f'Data/EUPs/{file}.txt')
        raw = EUP_f.read()
        EUP_f.close()
        raw_EUP = raw.replace('nan', "'nan'").replace('array', 'np.array')
        EUP = eval(raw_EUP)
    
        if EUP['InSAR'] != -999:
            for keykey in [k for k in EUP['InSAR'].keys() if k[0] == 'h']:
                for d in range(len(EUP['InSAR'][keykey])):
                    if EUP['InSAR'][keykey][d] == 'nan':
                        EUP['InSAR'][keykey][d] = np.nan
    
        xI = EUP['xI']
        xP = EUP['xP']
        xE = EUP['xE']
        h0 = EUP['h0']
        tau = EUP['tau']
    else:
        xI = file["xI"]
        xP = file["xP"]
        xE = file["xE"]
        h0 = 0
        tau = file["tau"]

    tt = []
    h = []
    cur_t = t_eval_min
    while get_yearfrac('{:0>4d}-{:0>2d}-{:0>2d}'.format(cur_t.year, cur_t.month, cur_t.day)) < tmax_yf:
        tt.append('{:0>4d}-{:0>2d}-{:0>2d}'.format(cur_t.year, cur_t.month, cur_t.day))
        cur_t += dt.timedelta(days=1)

    ndays = (tmax_dt-tmin_dt).days

    R_array = np.zeros((ndays, ))
    I_array = np.zeros((ndays, ))
    t_array = np.zeros((ndays, ))
    t0_idx = None

    for day in range(ndays):
        for backwards_day in range(int(tau)+1):
            cur_day = tmin_dt + dt.timedelta(days=day-backwards_day)
            try:
                Pt = etmgeg[cur_day][0]
                Et = etmgeg[cur_day][1]
                if backwards_day == int(tau):
                    R_array[day] += (Pt*xP - Et*xE) * (tau % 1)
                else:
                    R_array[day] += (Pt*xP - Et*xE)

            except KeyError:
                continue

        if R_array[day] <= 0:
            I_array[day] = I_array[day-1] + xI
        else:
            I_array[day] = I_array[day-1]

        date = tmin_dt + dt.timedelta(days=day)
        t_array[day] = get_yearfrac('{:0>4d}-{:0>2d}-{:0>2d}'.format(date.year, date.month, date.day))
        if date == t0_dt:
            t0_idx = day

    t_lst = list(t_array)
    if t0_idx is None:
        raise ValueError('t0 > tmax!')

    xIfrac = sum(1*R_array<=0)/R_array.shape[0]

    R_array -= R_array[t0_idx]
    I_array -= I_array[t0_idx]


    for t in tt:
        if get_yearfrac(t) < tmin_yf:
            elev = h0 + R_array[0] + I_array[0] + (dt.datetime.strptime(t, '%Y-%m-%d') - tmin_dt).days * xI * xIfrac
        else:
            tidx = t_lst.index(get_yearfrac(t))
            elev = h0 + R_array[tidx]+I_array[tidx]
        h.append(elev)

    return [get_yearfrac(t) for t in tt], h

def convert_value_to_hex(value, vmin=-10, vmax=10):
    pct = min(max((value - vmin) / (vmax - vmin), 0), 1)
    clr = (int(255 * min(1, 2 - 2 * pct)), 
           int(255 * (1 - 2 * abs(0.5 - pct))), 
           int(255 * min(1, 2 * pct)))
    #clr = (int(255 * max(0, (1 - 2 * pct))), 0, (int(255 * max(2*pct - 1, 0))))
    hx = '#'
    for c in clr:
        hx += "{:0>2s}".format(hex(c)[2:])
    return hx

def map_index(EUP):
    """
    This function maps the index of the gpkg EUP to the txt EUP, as these are not the same:
    the gpkg maintains the original numbering before ~50000 EUPs were removed, the txt EUP
    starts counting from 0 without gaps
    :param EUP: the gpkg index of the EUP
    :return: the txt index of the EUP
    """
    keys = list(gpkg_data.keys())
    # print(keys.index(EUP))
    return keys.index(EUP)

def map_vis_index(EUP):
    """
    This function maps the index of the gpkg EUP to the visualisation EUP, as these are not the same:
    the gpkg maintains the original numbering before ~50000 EUPs were removed, the visualisation EUP
    starts counting from 0 without gaps and has EUPs with no data removed
    :param EUP: the gpkg index of the EUP
    :return: the vis index of the EUP
    """ 
    if EUP in GPKG_VIS_MAP.keys():
        return GPKG_VIS_MAP[EUP]
    vis_id = None
    start_id = map_index(EUP)
    for att_id in range(start_id, 0, -1):
        if att_id in vis_data.keys():
            if gpkg_data[EUP]['crd_dec'] == vis_data[att_id]['crd_dec']:
                vis_id = att_id
                break
    GPKG_VIS_MAP[EUP] = vis_id
    f = open("Data/gpkg_vis_map.txt", "w")
    f.write(str(GPKG_VIS_MAP))
    f.close()
    return vis_id

def convert_gpkg_to_wgs84(gpkg):
    for EUP in gpkg.keys():
        gpkg[EUP]['crd_wgs'] = convert_rd_to_wgs84(gpkg_data[EUP]['crd_dec'][0])
    return gpkg


In [ ]:
gpkg_data = read_gpkg('InSAR_ALS_GNSS_Grav_Leveling.gpkg')

In [ ]:
vis_data = read_visualisation_gpkg('Visualisation_export.gpkg')

In [ ]:
vis_data = add_vis_gpkg_parameters(vis_data, gpkg_data)

In [ ]:
parquets = glob.glob("Data/SPAMS10/*.parquet")
nkeys = 0
spams10_data = {}
for parquet in parquets:
    pqt_data = read_spams10(parquet, f'''Data/SPAMS10_processed/{parquet.split("/")[-1].split(".")[0]}.txt''')
    for key in pqt_data.keys():
        spams10_data[nkeys] = {}
        for key2 in pqt_data[key].keys():
            spams10_data[nkeys][key2] = pqt_data[key][key2]
        nkeys += 1


In [ ]:
gpkg_data = convert_gpkg_to_wgs84(gpkg_data)

In [ ]:
output = widgets.Output()

out1 = widgets.Output()
out2 = widgets.Output()
out3 = widgets.Output()
out4 = widgets.Output()
out5 = widgets.Output()
out6 = widgets.Output()

tab = widgets.Tab(children = [out1, out2])
tab.set_title(0, 'Estimated Model')
tab.set_title(1, 'Unused')

tab2 = widgets.Tab(children = [out3, out4])
tab2.set_title(0, 'Parameters')
tab2.set_title(1, 'Unused')

tab3 = widgets.Tab(children = [out5])
tab3.set_title(0, 'Map colorbar')

tab4 = widgets.Tab(children = [out6])
tab4.set_title(0, "Search bar")

In [ ]:
CarDB = basemap_to_tiles(basemaps.CartoDB.Positron)
CarDB.base = True
CarDB.name = 'Map (CartoDB)'

m = Map(center=list(np.mean(np.array(aoi_wgs_poly), axis=0)), zoom=12, min_zoom=12, close_popup_on_click=False , layers = [CarDB])

AoI = LayerGroup(name = 'AoI')        

line = Polyline(locations = aoi_wgs_poly,
            weight = 2,
            color= '#000000', linestyle='--', fill=False) 

highlighter = Polyline(locations = [[0, 0], [0, 0]],
                       weight=3,
                       color='#5abf4d', fill=False, name='Selection')

AoI.add_layer(line)
highlighter_group = LayerGroup(name="highlighter")
highlighter_group.add_layer(highlighter)

In [ ]:
def plot_data(EUP, m):
    def button_click(**kwargs):
        # clear_output(wait=True)
        if isinstance(EUP, str):
            # it's spams10
            spams10_EUP = eval(EUP.split("_")[1])
            data = {"model_mode": "SPAMS_default"}
            highlighter.locations = [[spams10_data[spams10_EUP]["pnt_crd"][1]+3e-4, spams10_data[spams10_EUP]["pnt_crd"][0]+3e-4],
                                     [spams10_data[spams10_EUP]["pnt_crd"][1]-3e-4, spams10_data[spams10_EUP]["pnt_crd"][0]+3e-4],
                                     [spams10_data[spams10_EUP]["pnt_crd"][1]-3e-4, spams10_data[spams10_EUP]["pnt_crd"][0]-3e-4],
                                     [spams10_data[spams10_EUP]["pnt_crd"][1]+3e-4, spams10_data[spams10_EUP]["pnt_crd"][0]-3e-4],
                                     [spams10_data[spams10_EUP]["pnt_crd"][1]+3e-4, spams10_data[spams10_EUP]["pnt_crd"][0]+3e-4]]
            t, h = plot_model({"xI": spams10_data[spams10_EUP]["xI_mpy"] / 365.2425,
                               "xP": spams10_data[spams10_EUP]["xP_mpmm"],
                               "xE": spams10_data[spams10_EUP]["xE_mpmm"],
                               "tau": spams10_data[spams10_EUP]["tau_kd"] * 1000},
                              mode="SPAMS10")
        else:
            txt_eup = map_index(EUP)
            vis_eup = map_vis_index(EUP)                        
            t, h = plot_model(txt_eup)
            y, yt, ys = get_data(txt_eup)
            EUP_f = open(f'Data/EUPs/{txt_eup}.txt')
            raw = EUP_f.read()
            EUP_f.close()
            raw_EUP = raw.replace('nan', "'nan'").replace('array', 'np.array')
            data = eval(raw_EUP)
            highlighter.locations = convert_rd_to_wgs84(gpkg_data[EUP]['crd_dec'][0])
    
            yhat = [h[t.index(yt_)] for yt_ in yt]
            ehat = np.array(yhat) - np.array(y)
            T_stat = np.sum(ehat ** 2 / np.array(ys) ** 2)
            if vis_eup is not None:
                if data["model_mode"] == "SPAMS":
                    nu = int(1000*vis_data[vis_eup]['n_observations_1000']) - 5
                else:
                    nu = int(1000*vis_data[vis_eup]['n_observations_1000']) - 2
                T_stat_norm = T_stat / nu
        
        with out1:
            try:
                plt.close(fig="Spatial unit timeseries")
            except:
                pass
            clear_output(wait=True)
            if data["model_mode"] != "SPAMS_default":
                print("Red lines indicate 1-sigma standard deviation")
            plt.figure(figsize=(11,5), num="Spatial unit timeseries")
            plt.plot(t, h, label='Model fit')
            if data["model_mode"] != "SPAMS_default":
                plt.errorbar(yt, y, yerr=ys, fmt='o', ecolor='r', label='Observations')
            plt.grid(True)
            plt.xlabel('Time [y]')
            if data["model_mode"] == "SPAMS_default":
                plt.ylabel('Elevation change\nw.r.t. 1 Jan 2015 [m]')
            else:
                plt.ylabel('Elevation [m - NAP]')
            plt.legend()
            plt.show()
        with out3:
            clear_output(wait=True)
            if data["model_mode"] != "SPAMS_default":
                if vis_eup is None:
                    print('This spatial unit was filtered out of the resulting dataset due to a failed model estimate')
                else:
                    nsigfig_xi_raw = np.log10(vis_data[vis_eup]['sigma_xI_mpy']*1000)
                    if nsigfig_xi_raw < 0:
                        nsigfig = int(np.ceil(-1 * nsigfig_xi_raw) + 1)
                        xi_disp = round(vis_data[vis_eup]['xI_mpy']*1000, nsigfig)
                        sxi_disp = round(vis_data[vis_eup]['sigma_xI_mpy']*1000, nsigfig)
                        msr_disp = round(vis_data[vis_eup]['mean_yearly_irreversible_subsidence_MYIS_mpy']*1000, nsigfig)
                        smsr_disp = round(vis_data[vis_eup]['sigma_MYIS_mpy']*1000, nsigfig)
                    elif nsigfig_xi_raw < 1:
                        xi_disp = round(vis_data[vis_eup]['xI_mpy']*1000, 1)
                        sxi_disp = round(vis_data[vis_eup]['sigma_xI_mpy']*1000, 1)
                        msr_disp = round(vis_data[vis_eup]['mean_yearly_irreversible_subsidence_MYIS_mpy']*1000, 1)
                        smsr_disp = round(vis_data[vis_eup]['sigma_MYIS_mpy']*1000, 1)
                    else:
                        xi_disp = round(vis_data[vis_eup]['xI_mpy']*1000, 0)
                        sxi_disp = round(vis_data[vis_eup]['sigma_xI_mpy']*1000, 0)  
                        msr_disp = round(vis_data[vis_eup]['mean_yearly_irreversible_subsidence_MYIS_mpy']*1000, 0)
                        smsr_disp = round(vis_data[vis_eup]['sigma_MYIS_mpy']*1000, 0)
    
                    if data['model_mode'] in ["SPAMS", "SPAMS_default"]:
                        nsigfig_xp_raw = np.log10(vis_data[vis_eup]['sigma_xP_mpmm']*1000)
                        if nsigfig_xp_raw < 0:
                            nsigfig = int(np.ceil(-1 * nsigfig_xp_raw) + 1)
                            xp_disp = round(vis_data[vis_eup]['xP_mpmm']*1000, nsigfig)
                            sxp_disp = round(vis_data[vis_eup]['sigma_xP_mpmm']*1000, nsigfig)
                        elif nsigfig_xp_raw < 1:
                            xp_disp = round(vis_data[vis_eup]['xP_mpmm']*1000, 1)
                            sxp_disp = round(vis_data[vis_eup]['sigma_xP_mpmm']*1000, 1)
                        else:
                            xp_disp = round(vis_data[vis_eup]['xP_mpmm']*1000, 0)
                            sxp_disp = round(vis_data[vis_eup]['sigma_xP_mpmm']*1000, 0)  
    
                        nsigfig_xe_raw = np.log10(vis_data[vis_eup]['sigma_xE_mpmm']*1000)
                        if nsigfig_xe_raw < 0:
                            nsigfig = int(np.ceil(-1 * nsigfig_xe_raw) + 1)
                            xe_disp = round(vis_data[vis_eup]['xE_mpmm']*1000, nsigfig)
                            sxe_disp = round(vis_data[vis_eup]['sigma_xE_mpmm']*1000, nsigfig)
                        elif nsigfig_xe_raw < 1:
                            xe_disp = round(vis_data[vis_eup]['xE_mpmm']*1000, 1)
                            sxe_disp = round(vis_data[vis_eup]['sigma_xE_mpmm']*1000, 1)
                        else:
                            xe_disp = round(vis_data[vis_eup]['xE_mpmm']*1000, 0)
                            sxe_disp = round(vis_data[vis_eup]['sigma_xE_mpmm']*1000, 0) 
                        if data["model_mode"] == "SPAMS":
                            print(f'''GPKG spatial unit {EUP} (TXT {txt_eup}, VIS {vis_eup})
----------
Estimated parameters ({data['model_mode'] + ('-extended' if data['model_mode'] == 'SPAMS' else '')} model)

Elevation (1 Jan 2015): {round(vis_data[vis_eup]['h2015_m'], 3)}±{round(vis_data[vis_eup]['sigma_h2015_m'], 3)} m - NAP
XI (irreversible): {xi_disp}±{sxi_disp} mm/y ({round(vis_data[vis_eup]['xI_frac_1000pct']*1000, 1)}% active)
XP (precipitation): {xp_disp}±{sxp_disp} mm/mm
XE (evapotranspiration): {xe_disp}±{sxe_disp} mm/mm
Tau (delay): {round(vis_data[vis_eup]['tau_kd']*1000, 2)}±{round(vis_data[vis_eup]['sigma_tau_kd']*1000, 2)} days

----------
Derived variables

Mean Subsidence Rate: {msr_disp}±{smsr_disp} mm/y 
Observation timespan: {round(vis_data[vis_eup]['timespan_ky']*1000, 3)} y ({round(vis_data[vis_eup]['oldest_observation_ky']*1000, 3)} - {round(vis_data[vis_eup]['oldest_observation_ky']*1000 + vis_data[vis_eup]['timespan_ky']*1000, 3)})
# observations: {int(1000*vis_data[vis_eup]['n_observations_1000'])}

----------
Variances
var(h0): {vis_data[vis_eup]['sigma_h2015_m']**2:.2E} m^2
var(xI): {(vis_data[vis_eup]['sigma_xI_mpy']*1000)**2:.2E} mm^2/y^2
var(xP): {(vis_data[vis_eup]['sigma_xP_mpmm']*1000)**2:.2E} mm^2/mm^2
var(xE): {(vis_data[vis_eup]['sigma_xE_mpmm']*1000)**2:.2E} mm^2/mm^2
var(tau): {(vis_data[vis_eup]['sigma_tau_kd']*1000)**2:.2E} days^2

Covariances
cov(h0, xI): {vis_data[vis_eup]['rho_h2015_xI_1000']*1000 * vis_data[vis_eup]['sigma_h2015_m'] * vis_data[vis_eup]['sigma_xI_mpy']*1000:.2E}
cov(h0, xP): {vis_data[vis_eup]['rho_h2015_xP_1000']*1000 * vis_data[vis_eup]['sigma_h2015_m'] * vis_data[vis_eup]['sigma_xP_mpmm']*1000:.2E}
cov(h0, xE): {vis_data[vis_eup]['rho_h2015_xE_1000']*1000 * vis_data[vis_eup]['sigma_h2015_m'] * vis_data[vis_eup]['sigma_xE_mpmm']*1000:.2E}
cov(h0, tau): {vis_data[vis_eup]['rho_h2015_tau_1000']*1000 * vis_data[vis_eup]['sigma_h2015_m'] * vis_data[vis_eup]['sigma_tau_kd']*1000:.2E}
cov(xI, xE): {vis_data[vis_eup]['rho_xI_xE_1000']*1000 * vis_data[vis_eup]['sigma_xI_mpy']*1000 * vis_data[vis_eup]['sigma_xE_mpmm']*1000:.2E}
cov(xI, xP): {vis_data[vis_eup]['rho_xI_xP_1000']*1000 * vis_data[vis_eup]['sigma_xI_mpy']*1000 * vis_data[vis_eup]['sigma_xP_mpmm']*1000:.2E}
cov(xI, tau): {vis_data[vis_eup]['rho_xI_tau_1000']*1000 * vis_data[vis_eup]['sigma_xI_mpy']*1000 * vis_data[vis_eup]['sigma_tau_kd']*1000:.2E}
cov(xP, xE): {vis_data[vis_eup]['rho_xP_xE_1000']*1000 * vis_data[vis_eup]['sigma_xP_mpmm']*1000 * vis_data[vis_eup]['sigma_xE_mpmm']*1000:.2E}
cov(xP, tau): {vis_data[vis_eup]['rho_xP_tau_1000']*1000 * vis_data[vis_eup]['sigma_xP_mpmm']*1000 * vis_data[vis_eup]['sigma_tau_kd']*1000:.2E}
cov(xE, tau): {vis_data[vis_eup]['rho_xE_tau_1000']*1000 * vis_data[vis_eup]['sigma_xE_mpmm']*1000 * vis_data[vis_eup]['sigma_tau_kd']*1000:.2E}

---------
Statistics

Significant mean subsidence rate detected: {"No" if vis_data[vis_eup]['significant_irreversible_subsidence_detected_2s_1000'] < 0.00001 else "Yes"} (alpha=0.05) / {"No" if vis_data[vis_eup]['significant_irreversible_subsidence_detected_3s_1000'] < 0.00001 else "Yes"} (alpha=0.01)
Chosen mathematical model fits observations: {"No" if vis_data[vis_eup]['OMT_sust_2s_1000'] < 0.00001 else "Yes"} (alpha=0.05) / {"No" if vis_data[vis_eup]['OMT_sust_3s_1000'] < 0.00001 else "Yes"} (alpha=0.01) | Normalised F-value (T/nu) = {T_stat_norm:.2E}
Discriminatory power of Mean Subsidence Rate: {round(vis_data[vis_eup]['discriminatory_power_2s_1000pct']*1000, 2)}% (alpha=0.05) / {round(vis_data[vis_eup]['discriminatory_power_3s_1000pct']*1000, 2)}% (alpha=0.01)
Minimum Detectable Displacement Rate (80% confidence): {vis_data[vis_eup]['MDD_80p_mpy']*1000:.2E} mm/y

---------
Map link
https://www.topotijdreis.nl/satelliet/2024/@{vis_data[vis_eup]["center_dec"][0]},{vis_data[vis_eup]["center_dec"][1]},11

''')                            
                    else: 
                        print(f'''GPKG spatial unit {EUP} (TXT {txt_eup}, VIS {vis_eup})
----------
Estimated parameters ({data['model_mode'] + ('-extended' if data['model_mode'] == 'SPAMS' else '')} model)

Elevation (1 Jan 2015): {round(vis_data[vis_eup]['h2015_m'], 3)}±{round(vis_data[vis_eup]['sigma_h2015_m'], 3)} m - NAP
XI (irreversible): {xi_disp}±{sxi_disp} mm/y ({round(vis_data[vis_eup]['xI_frac_1000pct']*1000, 1)}% active)
----------
Derived variables

Mean Subsidence Rate: {msr_disp}±{smsr_disp} mm/y 
Observation timespan: {round(vis_data[vis_eup]['timespan_ky']*1000, 3)} y ({round(vis_data[vis_eup]['oldest_observation_ky']*1000, 3)} - {round(vis_data[vis_eup]['oldest_observation_ky']*1000 + vis_data[vis_eup]['timespan_ky']*1000, 3)})
# observations: {int(1000*vis_data[vis_eup]['n_observations_1000'])}

----------
Variances
var(h0): {vis_data[vis_eup]['sigma_h2015_m']**2:.2E} m^2
var(xI): {(vis_data[vis_eup]['sigma_xI_mpy']*1000)**2:.2E} mm^2/y^2

Covariances
cov(h0, xI): {vis_data[vis_eup]['rho_h2015_xI_1000']*1000 * vis_data[vis_eup]['sigma_h2015_m'] * vis_data[vis_eup]['sigma_xI_mpy']*1000:.2E}

---------
Statistics

Significant mean subsidence rate detected: {"No" if vis_data[vis_eup]['significant_irreversible_subsidence_detected_2s_1000'] < 0.00001 else "Yes"} (alpha=0.05) / {"No" if vis_data[vis_eup]['significant_irreversible_subsidence_detected_3s_1000'] < 0.00001 else "Yes"} (alpha=0.01)
Chosen mathematical model fits observations: {"No" if vis_data[vis_eup]['OMT_sust_2s_1000'] < 0.00001 else "Yes"} (alpha=0.05) / {"No" if vis_data[vis_eup]['OMT_sust_3s_1000'] < 0.00001 else "Yes"} (alpha=0.01) | Normalised F-value (T/nu) = {T_stat_norm:.2E}
Discriminatory power of Mean Subsidence Rate: {round(vis_data[vis_eup]['discriminatory_power_2s_1000pct']*1000, 2)}% (alpha=0.05) / {round(vis_data[vis_eup]['discriminatory_power_3s_1000pct']*1000, 2)}% (alpha=0.01)
Minimum Detectable Displacement Rate (80% confidence): {vis_data[vis_eup]['MDD_80p_mpy']*1000:.2E} mm/y

---------
Map link
https://www.topotijdreis.nl/satelliet/2024/@{vis_data[vis_eup]["center_dec"][0]},{vis_data[vis_eup]["center_dec"][1]},11
''')
            else:
                nsigfig_xi_raw = np.log10(spams10_data[spams10_EUP]['sigma_xI_mpy']*1000)
                if nsigfig_xi_raw < 0:
                    nsigfig = int(np.ceil(-1 * nsigfig_xi_raw) + 1)
                    xi_disp = round(spams10_data[spams10_EUP]['xI_mpy']*1000, nsigfig)
                    sxi_disp = round(spams10_data[spams10_EUP]['sigma_xI_mpy']*1000, nsigfig)
                    msr_disp = round(spams10_data[spams10_EUP]['mean_yearly_irreversible_subsidence_MYIS_mpy']*1000, nsigfig)
                    smsr_disp = round(spams10_data[spams10_EUP]['sigma_MYIS_mpy']*1000, nsigfig)
                elif nsigfig_xi_raw < 1:
                    xi_disp = round(spams10_data[spams10_EUP]['xI_mpy']*1000, 1)
                    sxi_disp = round(spams10_data[spams10_EUP]['sigma_xI_mpy']*1000, 1)
                    msr_disp = round(spams10_data[spams10_EUP]['mean_yearly_irreversible_subsidence_MYIS_mpy']*1000, 1)
                    smsr_disp = round(spams10_data[spams10_EUP]['sigma_MYIS_mpy']*1000, 1)
                else:
                    xi_disp = round(spams10_data[spams10_EUP]['xI_mpy']*1000, 0)
                    sxi_disp = round(spams10_data[spams10_EUP]['sigma_xI_mpy']*1000, 0)  
                    msr_disp = round(spams10_data[spams10_EUP]['mean_yearly_irreversible_subsidence_MYIS_mpy']*1000, 0)
                    smsr_disp = round(spams10_data[spams10_EUP]['sigma_MYIS_mpy']*1000, 0)

                nsigfig_xp_raw = np.log10(spams10_data[spams10_EUP]['sigma_xP_mpmm']*1000)
                if nsigfig_xp_raw < 0:
                    nsigfig = int(np.ceil(-1 * nsigfig_xp_raw) + 1)
                    xp_disp = round(spams10_data[spams10_EUP]['xP_mpmm']*1000, nsigfig)
                    sxp_disp = round(spams10_data[spams10_EUP]['sigma_xP_mpmm']*1000, nsigfig)
                elif nsigfig_xp_raw < 1:
                    xp_disp = round(spams10_data[spams10_EUP]['xP_mpmm']*1000, 1)
                    sxp_disp = round(spams10_data[spams10_EUP]['sigma_xP_mpmm']*1000, 1)
                else:
                    xp_disp = round(spams10_data[spams10_EUP]['xP_mpmm']*1000, 0)
                    sxp_disp = round(spams10_data[spams10_EUP]['sigma_xP_mpmm']*1000, 0)  

                nsigfig_xe_raw = np.log10(spams10_data[spams10_EUP]['sigma_xE_mpmm']*1000)
                if nsigfig_xe_raw < 0:
                    nsigfig = int(np.ceil(-1 * nsigfig_xe_raw) + 1)
                    xe_disp = round(spams10_data[spams10_EUP]['xE_mpmm']*1000, nsigfig)
                    sxe_disp = round(spams10_data[spams10_EUP]['sigma_xE_mpmm']*1000, nsigfig)
                elif nsigfig_xe_raw < 1:
                    xe_disp = round(spams10_data[spams10_EUP]['xE_mpmm']*1000, 1)
                    sxe_disp = round(spams10_data[spams10_EUP]['sigma_xE_mpmm']*1000, 1)
                else:
                    xe_disp = round(spams10_data[spams10_EUP]['xE_mpmm']*1000, 0)
                    sxe_disp = round(spams10_data[spams10_EUP]['sigma_xE_mpmm']*1000, 0) 



                
                print(f'''SPAMS10 spatial unit {EUP}
----------
Estimated parameters ({data['model_mode'] + ('-extended' if data['model_mode'] == 'SPAMS' else '')} model)

XI (irreversible): {xi_disp}±{sxi_disp} mm/y ({round(spams10_data[spams10_EUP]['xI_frac_1000pct']*1000, 1)}% active)
XP (precipitation): {xp_disp}±{sxp_disp} mm/mm
XE (evapotranspiration): {xe_disp}±{sxe_disp} mm/mm

----------
Fixed parameter
Tau (delay): {round(spams10_data[spams10_EUP]['tau_kd']*1000, 2)} days

----------
Derived variables

Mean Subsidence Rate: {msr_disp}±{smsr_disp} mm/y 
Observation timespan: Unknown
# observations: {int(spams10_data[spams10_EUP]['n_observations_1000'])}

----------
Variances
var(xI): {(spams10_data[spams10_EUP]['sigma_xI_mpy']*1000)**2:.2E} mm^2/y^2
var(xP): {(spams10_data[spams10_EUP]['sigma_xP_mpmm']*1000)**2:.2E} mm^2/mm^2
var(xE): {(spams10_data[spams10_EUP]['sigma_xE_mpmm']*1000)**2:.2E} mm^2/mm^2

Covariances
cov(xI, xE): {spams10_data[spams10_EUP]['rho_xI_xE_1000']*1000 * spams10_data[spams10_EUP]['sigma_xI_mpy']*1000 * spams10_data[spams10_EUP]['sigma_xE_mpmm']*1000:.2E}
cov(xI, xP): {spams10_data[spams10_EUP]['rho_xI_xP_1000']*1000 * spams10_data[spams10_EUP]['sigma_xI_mpy']*1000 * spams10_data[spams10_EUP]['sigma_xP_mpmm']*1000:.2E}
cov(xP, xE): {spams10_data[spams10_EUP]['rho_xP_xE_1000']*1000 * spams10_data[spams10_EUP]['sigma_xP_mpmm']*1000 * spams10_data[spams10_EUP]['sigma_xE_mpmm']*1000:.2E}

---------
Statistics

RSS: {round(spams10_data[spams10_EUP]["rss"], 2)}

---------
Map link
https://www.topotijdreis.nl/satelliet/2024/@{spams10_data[spams10_EUP]["center_dec"][0]},{spams10_data[spams10_EUP]['center_dec'][1]},11
''')
            
    return button_click

In [ ]:
eup_idx = []
eup_polys = []
eup_layer_groups = [LayerGroup()]
ct = 0
MAX_GROUP_SIZE = 250

print("Adding D-DEM...")
for EUP in list(gpkg_data.keys()):
    try:
        vis_eup = map_vis_index(EUP)
        if vis_eup is None:
            continue

        polygon = Polygon(
            locations = gpkg_data[EUP]['crd_wgs'],
            weight=0,
            fill_color='black',
            fill_opacity=1)
        polygon.on_click(plot_data(EUP, m=m))
        eup_polys.append(polygon)
        eup_layer_groups[-1].add_layer(eup_polys[-1])
        eup_idx.append(vis_eup)
        ct += 1
        if ct % MAX_GROUP_SIZE == 0:
            m.add_layer(eup_layer_groups[-1])
            eup_layer_groups.append(LayerGroup())
        if ct % 1000 == 0:
            print(f"{ct}/{len(list(gpkg_data.keys()))} done...")
    except TypeError:
        pass

print("Adding SPAMS10...")
for spams10_EUP in list(spams10_data.keys()):
    polygon = Polygon(
        locations = [[spams10_data[spams10_EUP]["pnt_crd"][1]+3e-4, spams10_data[spams10_EUP]["pnt_crd"][0]+3e-4],
                     [spams10_data[spams10_EUP]["pnt_crd"][1]-3e-4, spams10_data[spams10_EUP]["pnt_crd"][0]+3e-4],
                     [spams10_data[spams10_EUP]["pnt_crd"][1]-3e-4, spams10_data[spams10_EUP]["pnt_crd"][0]-3e-4],
                     [spams10_data[spams10_EUP]["pnt_crd"][1]+3e-4, spams10_data[spams10_EUP]["pnt_crd"][0]-3e-4],
                     [spams10_data[spams10_EUP]["pnt_crd"][1]+3e-4, spams10_data[spams10_EUP]["pnt_crd"][0]+3e-4]],
        weight=1,
        color="black",
        fill_color='black',
        fill_opacity=1)
    polygon.on_click(plot_data(f"SPAMS10_{spams10_EUP}", m=m))
    eup_polys.append(polygon)
    eup_layer_groups[-1].add_layer(eup_polys[-1])
    eup_idx.append(f"SPAMS10_{spams10_EUP}")
    ct += 1
    if ct % MAX_GROUP_SIZE == 0:
        m.add_layer(eup_layer_groups[-1])
        eup_layer_groups.append(LayerGroup())
    if ct % 1000 == 0:
        print(f"{ct}/{len(list(gpkg_data.keys()))+len(list(spams10_data.keys()))} done...")

m.add_layer(eup_layer_groups[-1])

In [ ]:
m.add_layer(AoI)
m.add_layer(highlighter_group)

In [ ]:
len(eup_idx), len(eup_polys)

In [ ]:
def plot_overview(plot='Mean Subsidence Rate (MSR)'):
    highlighter.locations = [(0,0), (0,0)]
    #with output:
    #    clear_output(wait=True)
    #    display(main_plot)
    if plot != 'Select':
        vdir = {
            'Mean Subsidence Rate (MSR)': [-10, 10, "mm/y", "both"],
            'Mean Subsidence Rate Standard Deviation': [0, 10, "mm/y", "max"],
            'Elevation at 1 Jan 2015': [-8, 2, "m", "both"],
            'Elevation at 1 Jan 2015 Standard Deviation': [0, 10, "cm", "max"],
            'Number of Observations': [0, 20, "#", "max"],
            'Timespan': [0, 60, "y", "max"],
            'xI': [-10, 10, "mm/y", "both"],
            'xI Standard Deviation': [0, 10, "mm/y", "max"],
            'Irreversible subsidence active fraction': [0, 100, "%", "neither"],
            'xP': [0, 0.15, "mm/mm", "max"],
            'xP Standard Deviation': [0, 0.01, "mm/mm", "max"],
            'xE': [0, 0.15, "mm/mm", "max"],
            'xE Standard Deviation': [0, 0.01, "mm/mm", "max"],
            'Tau': [0, 100, "days", "max"],
            'Tau Standard Deviation': [0, 1, "days", "max"],
            'Significant MSR detected (alpha=0.05)': [0, 1, "No < > Yes", 'neither'], 
            'Significant MSR detected (alpha=0.01)': [0, 1, "No < > Yes", 'neither'], 
            'Model Fits Observations (alpha=0.05)': [0, 1, "No < > Yes", 'neither'], 
            'Model Fits Observations (alpha=0.01)': [0, 1, "No < > Yes", 'neither'], 
            'Model Fit Normalized F-value': [0.5, 1.5, "-", "both"],
            'MSR Discriminatory Power (alpha=0.05)': [0, 100, "%", 'neither'], 
            'MSR Discriminatory Power (alpha=0.01]':[0, 100, "%", 'neither'], 
            'Min Det. Displacement Rate (80% confidence)': [0, 2, "mm/y", "max"],
            "First observation": [1950, 2010, "y", "both"]
       }
        vis_layer_translations = {
            'Mean Subsidence Rate (MSR)': ['mean_yearly_irreversible_subsidence_MYIS_mpy', 1000],
            'Mean Subsidence Rate Standard Deviation': ['sigma_MYIS_mpy', 1000],
            'Elevation at 1 Jan 2015': ['h2015_m', 1],
            'Elevation at 1 Jan 2015 Standard Deviation': ['sigma_h2015_m', 100],
            'Number of Observations': ['n_observations_1000', 1000],
                'Timespan': ['timespan_kd', 1000],
                'xI': ['xI_mpy', 1000],
                'xI Standard Deviation': ['sigma_xI_mpy', 1000],
                'Irreversible subsidence active fraction': ['xI_frac_1000pct', 1000],
                'xP': ["xP_mpmm", 1000],
                'xP Standard Deviation': ["sigma_xP_mpmm", 1000],
                'xE': ["xE_mpmm", 1000],
                'xE Standard Deviation': ["sigma_xE_mpmm", 1000],
                'Tau': ["tau_kd", 1000],
                'Tau Standard Deviation': ["sigma_tau_kd", 1000],
                'Significant MSR detected (alpha=0.05)': ["significant_irreversible_subsidence_detected_2s_1000", 1000], 
                'Significant MSR detected (alpha=0.01)': ["significant_irreversible_subsidence_detected_3s_1000", 1000], 
                'Model Fits Observations (alpha=0.05)': ["OMT_sust_2s_1000", 1000], 
                'Model Fits Observations (alpha=0.01)': ["OMT_sust_3s_1000", 1000], 
                'MSR Discriminatory Power (alpha=0.05)': ["discriminatory_power_2s_1000pct", 1000], 
                'MSR Discriminatory Power (alpha=0.01]':["discriminatory_power_3s_1000pct", 1000], 
                'Model Fit Normalized F-value': ["F_value_norm", 1],
                'Min Det. Displacement Rate (80% confidence)': ["MDD_80p_mpy", 1000],
                'First observation': ["oldest_observation_ky", 1000],
        }
                    
        #with output:
        if True:
            
            def update_data(b):
                with out4:
                    print(f'Should remove {b.description.split(" ")[-1]}')

            #display(tab2)
            #display(tab)
            #display(tab3)
            
            ct = 0

            with out5:
                try:
                    plt.close(fig="Map colorbar")
                except:
                    pass
                clear_output(wait=True)
                fig, ax = plt.subplots(figsize=(10,2), num="Map colorbar")
                #ax.axis('off')
                colmap1 = mpl.colors.LinearSegmentedColormap.from_list("custom_gradient", ["#ff0000", "#ffffff", "#0000ff"])
                cbar = matplotlib.colorbar.ColorbarBase(ax, cmap=colmap1, orientation = 'horizontal',extend=vdir[plot][3], 
                                norm=mpl.colors.Normalize(vmin=vdir[plot][0],vmax=vdir[plot][1]))
                cbar.set_label(f"{plot} [{vdir[plot][2]}]")
                plt.tight_layout()
                plt.show()

            for n_eup, vis_eup in enumerate(eup_idx):
                if isinstance(vis_eup, str):
                    given_val = spams10_data[int(vis_eup.split("_")[1])][vis_layer_translations[plot][0]]
                    if given_val is not None:
                        val = given_val * vis_layer_translations[plot][1]
                        eup_polys[n_eup].fill_color = convert_value_to_hex(val, vdir[plot][0], vdir[plot][1])
                    else:
                        eup_polys[n_eup].fill_color = "#000000"
                else:                
                    val = vis_data[vis_eup][vis_layer_translations[plot][0]] * vis_layer_translations[plot][1]
                    eup_polys[n_eup].fill_color = convert_value_to_hex(val, vdir[plot][0], vdir[plot][1])
                if n_eup % 250 == 0:
                    with out1:
                        clear_output(wait=True)
                        print(f'Loading spatial units ({n_eup}/{len(eup_idx)})...')
                    t.sleep(0.25)  # otherwise the data rate becomes too high for the map to render
            
            with out1:
                clear_output(wait=True)
                print('Loaded spatial units!')
                print(f'Click to analyse a spatial unit')
            with out2:
                clear_output(wait=True)
                print('Loaded spatial units!')
                print(f'Click to analyse a spatial unit')
            with out3:
                clear_output(wait=True)
                print('Loaded spatial units!')
                print(f'Click to analyse a spatial unit')
            with out4:
                clear_output(wait=True)
                print('Loaded spatial units!')
                print(f'Click to analyse a spatial unit')

            #display(m)

In [ ]:
plot_drop = widgets.Dropdown(
        options=['Select', 
                 'Mean Subsidence Rate (MSR)', 
                 'Mean Subsidence Rate Standard Deviation', 
                 'Elevation at 1 Jan 2015', 
                 'Elevation at 1 Jan 2015 Standard Deviation',
                 'Number of Observations',
                'Timespan',
                'xI',
                'xI Standard Deviation',
                "Irreversible subsidence active fraction",
                'xP',
                'xP Standard Deviation',
                'xE',
                'xE Standard Deviation',
                'Tau',
                'Tau Standard Deviation',
                'Significant MSR detected (alpha=0.05)',
                'Significant MSR detected (alpha=0.01)',
                'Model Fits Observations (alpha=0.05)',
                'Model Fits Observations (alpha=0.01)',
                # 'Model Fit Normalized F-value',
                'MSR Discriminatory Power (alpha=0.05)',
                'MSR Discriminatory Power (alpha=0.01]',
                'Min Det. Displacement Rate (80% confidence)',
                 "First observation",
                ],
        value='Select',
        description='Parameter:',
        disabled=False,
    )
main_plot = interactive(plot_overview, plot=plot_drop)

In [ ]:
search_box_text = widgets.Text(
    placeholder="Enter Parcel Number (VIS / SPAMS10 ID)",
    disabled=False
)
search_button = widgets.Button(description="Search")

def search_click(b):
    if search_box_text.value.isnumeric():
        num = eval(search_box_text.value)

        with out6:
            clear_output(wait=True)
            print("Search on VIS ID (numbers) or SPAMS10-ID (SPAMS10_###)")
            display(search_box_text)
            display(search_button)
        if num in eup_idx:
            gpkg_num = None
            for key in GPKG_VIS_MAP.keys():
                if GPKG_VIS_MAP[key] == num:
                    gpkg_num = key
                    break
            if gpkg_num is not None:
                plot_data(gpkg_num, m=m)()
                txt_eup = map_index(gpkg_num)
                with out6:
                    print(f"Found VIS Spatial Unit {num} (GPKG ID {gpkg_num}, TXT ID {txt_eup})")
                EUP_f = open(f'Data/EUPs/{txt_eup}.txt')
                raw = EUP_f.read()
                EUP_f.close()
                raw_EUP = raw.replace('nan', "'nan'").replace('array', 'np.array')
                data = eval(raw_EUP)
                crds = convert_rd_to_wgs84(data['crd_dec'][0])
                mean_crds = [np.mean([i[0] for i in crds]), np.mean([i[1] for i in crds])]
                m.center = mean_crds
            else:
                with out6:
                    print("ERROR: VIS ID provided is invalid")
        else:
            with out6:
                print("ERROR: VIS ID provided is invalid")
    else:
        with out6:
            clear_output(wait=True)
            print("Search on VIS ID (numbers) or SPAMS10-ID (SPAMS10_###)")
            display(search_box_text)
            display(search_button)
        if search_box_text.value in eup_idx:
            plot_data(search_box_text.value, m=m)()
            spams10_id = eval(search_box_text.value.split("_")[1])
            m.center = [spams10_data[spams10_id]["pnt_crd"][1], spams10_data[spams10_id]["pnt_crd"][0]]
            with out6:
                print(f"Found SPAMS10 spatial unit {spams10_id}!")
        else:
            with out6:
                print(f"ERROR: SPAMS10 ID '{search_box_text.value}' is invalid. Make sure to follow the format SPAMS10_###")        

search_button.on_click(search_click)

with out6:
    print("Search on VIS ID (numbers) or SPAMS10-ID (SPAMS10_###)")
    display(search_box_text)
    display(search_button)


In [ ]:
%matplotlib ipympl
button = widgets.Button(description="Run Me!")


display(button, output)

def on_button_clicked(b):
    with output:
        clear_output()
        display(main_plot)
        
button.on_click(on_button_clicked)
display(tab4)
display(tab2)
display(tab)
display(tab3)
display(m)